In [533]:

import pandas as pd
import numpy as np
from util import *
from logistic_np import *
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Dense,InputLayer, LeakyReLU, BatchNormalization, Activation, Dropout, Input, Flatten, Reshape
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
# from EDA import myEDA
from keras.datasets import mnist
from keras.optimizers import Adamax, Adam, Nadam, SGD
from matplotlib import pyplot as plt

In [534]:
# define dataset
# X, y = make_classification(n_samples=1000, n_features=100, n_informative=10, n_redundant=90, random_state=1)
# # number of input columns
# n_inputs = X.shape[1]

# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

# X_train, y_train, x_val, y_val, X_test, y_test = get_mnist_data()


# X_train, y_train, X_test, y_test = get_vehicle_data() #For ehicle dataset
# X_train, val_x, y_train, val_y = train_test_split(X_train, y_train, test_size=0.33, random_state=1)
(X_train, y_train), (X_test, y_test) = mnist.load_data() #for hand writing
print(X_train.shape)
X_train, X_test = normalize_all_pixel(X_train,X_test)
# X_train, X_test = normalize_per_pixel(X_train,X_test)
X_train  = reshape2D(X_train)
# val_x = reshape2D(val_x)
X_test = reshape2D(X_test)
# scale data
# t = StandardScaler()
# t.fit(X_train)
# X_train = t.transform(X_train)
# X_test = t.transform(X_test)
print(X_train.shape[1])
encoder_dim = X_train.shape[1]//2
print(encoder_dim)
print(X_train.shape)
print(y_test)

(60000, 28, 28)
784
392
(60000, 784)
[7 2 1 ... 4 5 6]


In [535]:
Encoder = Sequential([
    InputLayer(input_shape=X_train.shape[1:]),
    Dense(encoder_dim),
    # Dropout(0.2),
    BatchNormalization(),
    LeakyReLU(),
    Dense(encoder_dim),
    Dense(encoder_dim//2)], name="Encoder")
Encoder.summary()



Model: "Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_318 (Dense)           (None, 392)               307720    
                                                                 
 batch_normalization_130 (Ba  (None, 392)              1568      
 tchNormalization)                                               
                                                                 
 leaky_re_lu_179 (LeakyReLU)  (None, 392)              0         
                                                                 
 dense_319 (Dense)           (None, 392)               154056    
                                                                 
 dense_320 (Dense)           (None, 196)               77028     
                                                                 
Total params: 540,372
Trainable params: 539,588
Non-trainable params: 784
___________________________________________________

In [536]:
Decoder = Sequential([
    InputLayer(input_shape=(encoder_dim//2,)),
    Dense(encoder_dim),
    BatchNormalization(),
    LeakyReLU(),
    # Dropout(0.2), 
    Dense(encoder_dim),
    Dense(X_train.shape[1])
    ], name="Decoder")
Decoder.summary()


Model: "Decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_321 (Dense)           (None, 392)               77224     
                                                                 
 batch_normalization_131 (Ba  (None, 392)              1568      
 tchNormalization)                                               
                                                                 
 leaky_re_lu_180 (LeakyReLU)  (None, 392)              0         
                                                                 
 dense_322 (Dense)           (None, 392)               154056    
                                                                 
 dense_323 (Dense)           (None, 784)               308112    
                                                                 
Total params: 540,960
Trainable params: 540,176
Non-trainable params: 784
___________________________________________________

In [537]:
inp = Input(X_train.shape[1:])
Encoderoutput = Encoder(inp)
Decoderoutput = Decoder(Encoderoutput)
AutoEncodermodel = Model(inputs = inp, outputs = Decoderoutput)
AutoEncodermodel.summary()
AutoEncodermodel.compile(optimizer = "adam", loss = "mse")

Model: "model_85"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_224 (InputLayer)      [(None, 784)]             0         
                                                                 
 Encoder (Sequential)        (None, 196)               540372    
                                                                 
 Decoder (Sequential)        (None, 784)               540960    
                                                                 
Total params: 1,081,332
Trainable params: 1,079,764
Non-trainable params: 1,568
_________________________________________________________________


In [538]:
# Train encoder model
callback = EarlyStopping(monitor='val_loss',patience=3)
hist = AutoEncodermodel.fit(X_train, X_train, epochs=100, validation_data=(X_test,X_test), callbacks=[callback])

Epoch 1/100
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1618 - val_loss: 0.1004
Epoch 2/100
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0848 - val_loss: 0.0781
Epoch 3/100
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0701 - val_loss: 0.0662
Epoch 4/100
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0646 - val_loss: 0.0671
Epoch 5/100
1875/1875 [==============================] - 20s 10ms/step - loss: 0.0623 - val_loss: 0.0659
Epoch 6/100
1875/1875 [==============================] - 20s 10ms/step - loss: 0.0611 - val_loss: 0.0693
Epoch 7/100
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0601 - val_loss: 0.0625
Epoch 8/100
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0591 - val_loss: 0.0602
Epoch 9/100
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0584 - val_loss: 0.0590
Epoch 10/100
1875/1875 [==============================] - 1

In [539]:
plt.plot(range(1, len(hist.history['loss']) + 1), hist.history['loss'], label="train")
plt.plot(range(1, len(hist.history['loss']) + 1),hist.history['val_loss'], label="test")
plt.legend()
plt.show()

In [540]:
encoder = Model(inputs=inp, outputs=Encoderoutput)
encoder.compile()
encoder.save('encoder.h5')

In [541]:
from keras.utils import plot_model

plot_model(plot_model(encoder, 'encoder_compress.png', show_shapes=True))

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


AttributeError: 'NoneType' object has no attribute 'built'

In [542]:
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
for i in range(100):
    gaussian.fit(X_train, y_train)
    print("Epoch: ",i )

y_pred = gaussian.predict(X_test)
from sklearn.metrics import accuracy_score
print("Accuracy: %.2f%%" %(100*accuracy_score(y_test, y_pred)))

Epoch:  0
Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15
Epoch:  16
Epoch:  17
Epoch:  18
Epoch:  19
Epoch:  20
Epoch:  21
Epoch:  22
Epoch:  23
Epoch:  24
Epoch:  25
Epoch:  26
Epoch:  27
Epoch:  28
Epoch:  29
Epoch:  30
Epoch:  31
Epoch:  32
Epoch:  33
Epoch:  34
Epoch:  35
Epoch:  36
Epoch:  37
Epoch:  38
Epoch:  39
Epoch:  40
Epoch:  41
Epoch:  42
Epoch:  43
Epoch:  44
Epoch:  45
Epoch:  46
Epoch:  47
Epoch:  48
Epoch:  49
Epoch:  50
Epoch:  51
Epoch:  52
Epoch:  53
Epoch:  54
Epoch:  55
Epoch:  56
Epoch:  57
Epoch:  58
Epoch:  59
Epoch:  60
Epoch:  61
Epoch:  62
Epoch:  63
Epoch:  64
Epoch:  65
Epoch:  66
Epoch:  67
Epoch:  68
Epoch:  69
Epoch:  70
Epoch:  71
Epoch:  72
Epoch:  73
Epoch:  74
Epoch:  75
Epoch:  76
Epoch:  77
Epoch:  78
Epoch:  79
Epoch:  80
Epoch:  81
Epoch:  82
Epoch:  83
Epoch:  84
Epoch:  85
Epoch:  86
Epoch:  87
Epoch:  88
Epoch:  89
Epoch:  90
Epoch:  9

In [543]:
from keras.models import load_model
feature_extract = load_model('encoder.h5')

In [544]:
X_train_encoded = feature_extract.predict(X_train)
X_test_encoded = feature_extract.predict(X_test)

313/313 [==============================] - 1s 4ms/step


In [545]:
gaussian_v2 = GaussianNB()
for i in range(300):
    gaussian_v2.fit(X_train_encoded, y_train)
    print("Epoch: ",i )

y_pred_v2 = gaussian_v2.predict(X_test_encoded)
from sklearn.metrics import accuracy_score
print("Accuracy: %.2f%%" %(100*accuracy_score(y_test, y_pred_v2)))

Epoch:  0
Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15
Epoch:  16
Epoch:  17
Epoch:  18
Epoch:  19
Epoch:  20
Epoch:  21
Epoch:  22
Epoch:  23
Epoch:  24
Epoch:  25
Epoch:  26
Epoch:  27
Epoch:  28
Epoch:  29
Epoch:  30
Epoch:  31
Epoch:  32
Epoch:  33
Epoch:  34
Epoch:  35
Epoch:  36
Epoch:  37
Epoch:  38
Epoch:  39
Epoch:  40
Epoch:  41
Epoch:  42
Epoch:  43
Epoch:  44
Epoch:  45
Epoch:  46
Epoch:  47
Epoch:  48
Epoch:  49
Epoch:  50
Epoch:  51
Epoch:  52
Epoch:  53
Epoch:  54
Epoch:  55
Epoch:  56
Epoch:  57
Epoch:  58
Epoch:  59
Epoch:  60
Epoch:  61
Epoch:  62
Epoch:  63
Epoch:  64
Epoch:  65
Epoch:  66
Epoch:  67
Epoch:  68
Epoch:  69
Epoch:  70
Epoch:  71
Epoch:  72
Epoch:  73
Epoch:  74
Epoch:  75
Epoch:  76
Epoch:  77
Epoch:  78
Epoch:  79
Epoch:  80
Epoch:  81
Epoch:  82
Epoch:  83
Epoch:  84
Epoch:  85
Epoch:  86
Epoch:  87
Epoch:  88
Epoch:  89
Epoch:  90
Epoch:  9

In [546]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# define model
model = LogisticRegression(multi_class="ovr")
# fit model on training set
model.fit(X_train, y_train)
# make prediction on test set
yhat = model.predict(X_test)
# calculate accuracy
acc = accuracy_score(y_test, yhat)
print(acc)

d:\anaconda\envs\py39\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7379


In [548]:
# define the model
model = LogisticRegression(multi_class="ovr")
# fit the model on the training set
model.fit(X_train_encoded, y_train)
# make predictions on the test set
yhat = model.predict(X_test_encoded)
acc = accuracy_score(y_test, yhat)
print(acc)

d:\anaconda\envs\py39\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6435
